# Introduction: Interferometry vs. photometry


Let's say we have a spherical star whose intensity map, projected into a 2D at a specific vieweing orientation, is defined as $I(x,y)$. Finding the total brightness of the star at that orientation is a sum over the visible surface of the star: 

 $$ F = \iint I(x,y)dx dy $$
 
 
 This problem of fiding the disk-integrated brightness of a star--the observable quantity of photometry--has been solved in the [starry package](https://arxiv.org/abs/1810.06559) in a rather elegant way that, among other things, permits a description of the information content of photometric data. This is done first by representing the surface map of a star using [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics). If the 3d surface of a star is represented using spherical harmonic coefficients $\mathbf{y}$, then we can write the intensity map as:
 
$$I(x,y) = \mathbf{\tilde{y}}^\top(x,y) \ \mathbf{R} \ \mathbf{y}$$

where $\mathbf{\tilde{y}}^\top(x,y)$ is the spherical harmonic basis, $\mathbf{R}$ is the rotation matrix into the correct viewing orientation with the viewer at $+\infty$ along the z axis and $\mathbf{y}$ is the vector of spherical harmonic coefficients. Because spherical harmonics form an orthonormal basis on a unit sphere, *any* map can be represented using a sufficiently high order expansion in the spherical harmonics. This makes it a natural choice to represent the surface of a star.

The real achievement of starry was to find an analytic way of performing the surface integral where a star is represented using spherical harmonics. This makes it extremely fast to compute photometric observables--light curves-- as a star rotates or even as a planet occults it. 

In this notebook, I will attempt to show that it is possible to use the same elegant description of the surface of a star in terms of spherical harmonics to find analytic observables used in interferometry. Recall that interferometric observations record a quantity called the visibility. The van-Cittert Zernike theorem relates the intensity map of a star to its visibility using a different kind of double integral from the one for photometry--the Fourier transform:

$$ V(u,v) = \iint I(x,y) e^{i(ux + vy)} dxdy $$

Where $V$ is the visibility at baseline (u,v). This integral is similar but not identical to the one from photometry, and a similar intuition should be able to help solve it. Remember that the spherical harmonic basis $\mathbf{\tilde{y}}^\top(x,y)$ is simply a polynomial, and therefore can be turned into a polynomial basis:

$$ \mathbf{\tilde{p}}^\top(x,y) = \mathbf{A} \ \mathbf{\tilde{y}}(x,y)$$


Next, we must find a way to take the Fourier transform of the surface map as a polynomial. Although this may be difficult in our polynomial basis, there is an ideal choice for a basis defined on a unit disk which has an analytic Fourier transform--the Zernikes. 

What we must do is find another change of basis matrix $\mathbf{B}$ for which

$$ \mathbf{\tilde{z}}^\top(x,y) = \mathbf{B} \ \mathbf{\tilde{p}}(x,y) $$

where $\mathbf{\tilde{z}}$ is the zernike basis. If we can find such a matrix $\mathbf{B}$, we can solve the Fourier integral as follows:

$$ V(u,v) = \mathbf{z}^\top \mathbf{B} \ \mathbf{A} \ \mathbf{y}$$

where $\mathbf{z}$ is the solution vector--the Fourier integral applied analytically to each term of the Zernike basis and written as a vector. 

In [4]:
import sympy as sp
from sympy import symbols, sin, cos, Matrix, Eq, Rational, floor, sqrt
from sympy import simplify, factorial, pi, binomial, factor, expand, collect
from sympy.functions.special.tensor_functions import KroneckerDelta
from sympy import init_printing
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pandas as pd
from sympy import latex
from scipy.optimize import curve_fit
from IPython.display import display, Math
from hswfs import zernike

Lets show the first 15 terms in the polynomial basis $ \mathbf{\tilde{p}}(x,y) $:

In [27]:
def poly_basis(n, x, y):
    """Return the n^th term in the polynomial basis."""
    l = Rational(floor(sqrt(n)))
    m = Rational(n - l * l - l)
    mu = Rational(l - m)
    nu = Rational(l + m)
    if nu % 2 == 0:
        i = Rational(mu, 2)
        j = Rational(nu, 2)
        k = Rational(0)
    else:
        i = Rational(mu - 1, 2)
        j = Rational(nu - 1, 2)
        k = Rational(1)
    return x ** i * y ** j * sqrt(1 - x ** 2 - y ** 2) ** k


# Compute the polynomial basis
x, y, z = sp.symbols('x, y, z')
basis = Matrix([poly_basis(n, x, y) for n in range(15)]).T

def Coefficient(expression, term):
    """Return the coefficient multiplying `term` in `expression`."""
    # Get the coefficient
    coeff = expression.coeff(term)
    if term==1:
        coeff = expression.subs(sqrt(1 - x ** 2 - y ** 2), 0).subs(x, 0).subs(y, 0)
    # Set any non-constants in this coefficient to zero. If the coefficient
    # is not a constant, this is not the term we are interested in!
    coeff = coeff.subs(sqrt(1 - x ** 2 - y ** 2), 0).subs(x, 0).subs(y, 0)
    return coeff

pbasis = Matrix([term for term in basis])
pbasis

Matrix([
[                          1],
[                          x],
[     sqrt(-x**2 - y**2 + 1)],
[                          y],
[                       x**2],
[   x*sqrt(-x**2 - y**2 + 1)],
[                        x*y],
[   y*sqrt(-x**2 - y**2 + 1)],
[                       y**2],
[                       x**3],
[x**2*sqrt(-x**2 - y**2 + 1)],
[                     x**2*y],
[ x*y*sqrt(-x**2 - y**2 + 1)],
[                     x*y**2],
[y**2*sqrt(-x**2 - y**2 + 1)]])

Now the first 15 terms in the zernike basis $\mathbf{\tilde{z}}(x,y)$:

In [28]:
zbasis = Matrix([zernike.ZernikePolynomial(j=j).cartesian.simplify() for j in range(15)])
zbasis

Matrix([
[                                                                           1],
[                                                                         2*y],
[                                                                         2*x],
[                                                               2*sqrt(6)*x*y],
[                                               sqrt(3)*(2*x**2 + 2*y**2 - 1)],
[                                                       sqrt(6)*(x**2 - y**2)],
[                           2*sqrt(2)*(x**2 + y**2)**(3/2)*sin(3*atan2(y, x))],
[                                           2*sqrt(2)*y*(3*x**2 + 3*y**2 - 2)],
[                                           2*sqrt(2)*x*(3*x**2 + 3*y**2 - 2)],
[                           2*sqrt(2)*(x**2 + y**2)**(3/2)*cos(3*atan2(y, x))],
[                                                4*sqrt(10)*x*y*(x**2 - y**2)],
[                                        2*sqrt(10)*x*y*(4*x**2 + 4*y**2 - 3)],
[                         sqrt(

## Change of basis matrix:

Now lets write a single term in the zernike basis into the polynomial basis. Writing these vectors as the columns of a matrix for each term in the zernike basis should give us our change of basis matrix (or is it the inverse??):

In [43]:
vec = Matrix([Coefficient(sp.expand(zernike.ZernikePolynomial(j=4).cartesian), term) for term in basis])
vec

Matrix([
[ -sqrt(3)],
[        0],
[        0],
[        0],
[2*sqrt(3)],
[        0],
[        0],
[        0],
[2*sqrt(3)],
[        0],
[        0],
[        0],
[        0],
[        0],
[        0]])